In [1]:
import mujoco
import numpy as np
import pinocchio as pin
from robot_descriptions.skydio_x2_description import URDF_PATH
from robot_descriptions.skydio_x2_mj_description import MJCF_PATH
# from robot_descriptions.z1_description import URDF_PATH
# from robot_descriptions.z1_mj_description import MJCF_PATH

from mujoco_sysid import parameters, regressors
from mujoco_sysid.utils import muj2pin

In [2]:
mjmodel = mujoco.MjModel.from_xml_path(MJCF_PATH)
# enable energy
mjmodel.opt.enableflags |= mujoco.mjtEnableBit.mjENBL_ENERGY
# disable friction
mjmodel.opt.disableflags |= (
    mujoco.mjtDisableBit.mjDSBL_CONTACT | mujoco.mjtDisableBit.mjDSBL_FRICTIONLOSS | mujoco.mjtDisableBit.mjDSBL_LIMIT
)
mjdata = mujoco.MjData(mjmodel)

pinmodel = pin.buildModelFromUrdf(URDF_PATH)
pindata = pin.Data(pinmodel)

In [3]:
# np.random.seed(10)

In [4]:
q, v = np.random.randn(mjmodel.nq), np.random.randn(mjmodel.nv)


pinq, pinv = muj2pin(q, v)

In [5]:
mjdata.qpos[:] = q.copy()
mjdata.qvel[:] = v.copy()

mujoco.mj_forward(mjmodel, mjdata)

mjdata.energy, np.sum(mjdata.energy)

(array([3.30941522, 0.34777589]), 3.6571911151358094)

In [6]:
pin.computeAllTerms(pinmodel, pindata, pinq, pinv)

mjdata.xpos[-1], pindata.oMi[-1].translation

(array([-1.79547779,  1.10714143,  0.24186373]),
 array([-1.79547779,  1.10714143,  0.24186373]))

In [7]:
np.sum(
    [
        pin.computePotentialEnergy(pinmodel, pindata, pinq),
        pin.computeKineticEnergy(pinmodel, pindata, pinq, pinv),
    ]
)

2.8678604759610153

In [8]:
theta = np.concatenate([parameters.get_dynamic_parameters(mjmodel, i) for i in mjmodel.jnt_bodyid])

theta.shape, theta[:10]

((10,),
 array([ 1.325   ,  0.      ,  0.      ,  0.0715  ,  0.04051 ,  0.      ,
         0.02927 , -0.0021  ,  0.      ,  0.060528]))

In [9]:
params = []

for i in range(len(pinmodel.inertias) - 1):
    params.extend(pinmodel.inertias[i + 1].toDynamicParameters())

    last_params = np.array(params[-10:])
    last_theta = theta[i * 10 : (i + 1) * 10]

    # mass should match
    assert np.isclose(last_params[0], last_theta[0])

    # lever arm should match
    assert np.allclose(last_params[1:4], last_theta[1:4])

    print(f"for body {i} norm of difference is {np.linalg.norm(last_params - last_theta)}")

params = np.array(params)

params.shape, np.linalg.norm(params - theta)

for body 0 norm of difference is 1.511276715821191e-17


((10,), 1.511276715821191e-17)

In [10]:
reg_en = regressors.mj_energyRegressor(mjmodel, mjdata)[2]

reg_en @ theta, reg_en @ params

spatial [-0.27851279 -0.3888437  -0.48421291 -1.34814054 -0.21246592  1.05486061] [-1.34814054 -0.21246592  1.05486061 -0.27851279 -0.3888437  -0.48421291]


(3.657191115135809, 3.657191115135809)

In [11]:
def kinetic_regressor(q, v):
    pin.computeAllTerms(pinmodel, pindata, q, v)
    regressor = np.zeros((1, (pinmodel.nbodies - 1) * 10))
    for i in range(pinmodel.nbodies - 1):
        vel = pin.getVelocity(pinmodel, pindata, i + 1, pin.LOCAL)
        vl = vel.linear
        va = vel.angular
        print(vl, va)

        regressor[0, i * 10 + 0] = 0.5 * (vl[0] ** 2 + vl[1] ** 2 + vl[2] ** 2)
        regressor[0, i * 10 + 1] = -va[1] * vl[2] + va[2] * vl[1]
        regressor[0, i * 10 + 2] = va[0] * vl[2] - va[2] * vl[0]
        regressor[0, i * 10 + 3] = -va[0] * vl[1] + va[1] * vl[0]
        regressor[0, i * 10 + 4] = 0.5 * va[0] ** 2
        regressor[0, i * 10 + 5] = va[0] * va[1]
        regressor[0, i * 10 + 6] = 0.5 * va[1] ** 2
        regressor[0, i * 10 + 7] = va[0] * va[2]
        regressor[0, i * 10 + 8] = va[1] * va[2]
        regressor[0, i * 10 + 9] = 0.5 * va[2] ** 2

    return regressor

In [12]:
(
    kinetic_regressor(q, v) @ theta,
    kinetic_regressor(q, v) @ params,
)

[ 0.02826008 -0.38597266 -0.55987267] [-1.34814054 -0.21246592  1.05486061]
[ 0.02826008 -0.38597266 -0.55987267] [-1.34814054 -0.21246592  1.05486061]


(array([0.34339236]), array([0.34339236]))